# **Data Transform and Cleaning Process**

**Importing Libraries:**

In [1]:
import pandas as pd
import os
from etl_helper import extract_multi_csvs

**Merging all datasets into dataframe:**

In [2]:
# Creating variables for merging the datasets
directory_path = (r"C:\Users\user\OneDrive\RFA _Personal Files\02. COURSE\Purwadhika_Data Engineering\Purwadhika_VS\01_Personal_Projects\01_ETL01\data_original")
files_list = os.listdir(directory_path)

# Merging the datasets into one merged dataframe. Function extract_multi_csvs can refer to etl_helper.py file
df = extract_multi_csvs(directory_path,files_list)

**Checking the structure and details of the dataframe: columns, row data, datatypes, null values, etc):**

Based on the structure below, there are 5 things to do in transforming and cleaning the dataset:
1. ratings --> ensure it's a float dtype and have no string character inside X
2. no_of_ratings --> ensure it's a float dtype and have no string character inside
3. discount_price --> ensure it's a integer dtype have no string character inside
4. actual_price --> ensure it's a integer dtype have no string character inside
5. checking and handling null values within the dataframe
6. check for duplicated data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(1), object(9)
memory usage: 84.2+ MB


**Backup the original dataframe**:

In [4]:
#backup the original dataframe 
df1 = df.copy()
df1.head(5)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990",NaN
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990",NaN
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990",NaN
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990",NaN
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790",NaN


**Handling 'ratings' column:**

The "ratings" column should contain only numerical data. Therefore, it's important to ensure that no non-numeric characters appear in this column.

In [5]:
#ensure rating column is all numerical and non-numerical value. The errors=coerce will force the non-numerical value to change into NaN or null values
df1['ratings'] = pd.to_numeric(df1['ratings'], errors='coerce')

#convert null values to 0
df1['ratings'] = df1['ratings'].fillna(0)

**Handling 'no_of_ratings' column:**

This specific column is also should be treated the same way as 'ratings column', but since it represent number of ratings, it should be converted into integer.

In [6]:
#same treat as 'ratings' column
df1['no_of_ratings'] = pd.to_numeric(df1['no_of_ratings'].str.replace(",", ""),errors='coerce')
df1['no_of_ratings'] = df1['no_of_ratings'].fillna(0)

#convert datatype from float to Integer
df1['no_of_ratings'] = df1['no_of_ratings'].astype('Int64')


**Handling 'discount_price' and 'actual_price' columns:**

We've noticed that the 'discount_price' and 'actual_price' columns contain currency symbols. These columns should be converted to the float data type, removing any non-numeric characters.

In [7]:
#remove any non-numerical characters and fill NaN/null with 0
df1['discount_price'] = pd.to_numeric(df1['discount_price'].str.replace("₹","").str.replace(",",""), errors='coerce')
df1['discount_price'] = df1['discount_price'].fillna(0)

#same treat as 'discount_price'
df1['actual_price'] = pd.to_numeric(df1['actual_price'].str.replace("₹","").str.replace(",",""), errors='coerce')
df1['actual_price'] = df1['actual_price'].fillna(0)


**Handling unnecessary and missing values:**

We have converted the data to the desired types. Now, it’s time to proceed with data validation by addressing any missing values and removing unnecessary columns.
1. Handling the 'actual_price' column: The decision to remove this column is based on logic—it's unreasonable for a product listed on an online marketplace to lack an actual_price. Thus, it doesn't make sense to keep it.
2. Removing the unnecessary unnamed:0 column: This column provides no additional value for analysis and can be safely discarded.

In [8]:
# Keep only the data that does not only contain 0 in 'actual_price' column
df1 = df1[df1['actual_price'] != 0]


In [9]:
#remove the unecessary column that does not add value for analysis
df1 = df1.drop('Unnamed: 0', axis=1)

**Dataframe checkpoint**:
Further checking the dataframe

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067538 entries, 0 to 1103169
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1067538 non-null  object 
 1   main_category   1067538 non-null  object 
 2   sub_category    1067538 non-null  object 
 3   image           1067538 non-null  object 
 4   link            1067538 non-null  object 
 5   ratings         1067538 non-null  float64
 6   no_of_ratings   1067538 non-null  Int64  
 7   discount_price  1067538 non-null  float64
 8   actual_price    1067538 non-null  float64
dtypes: Int64(1), float64(3), object(5)
memory usage: 82.5+ MB


**Create new column named 'discount_amount':**

This specific column is to calculate the discount amount by subtracting 'actual_price' and 'discount_price'

In [11]:
#create new column using subtract operator
df1['discount_amount'] = df1['actual_price'] - df1['discount_price']

**Handling duplicated data within the dataframe:**

In [12]:
#checking how many rows are duplicated within the dataframe
df1.duplicated().sum()

533713

We observed 533,713 duplicate rows in the dataframe. It's essential to address these duplicates to ensure the analysis remains accurate and reliable.

In [13]:
#drop duplicate rows within the dataframe
df_cleaned_v1 = df1.drop_duplicates()

In [14]:
#further checking after remove duplicated data
df_cleaned_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 533825 entries, 0 to 951774
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   name             533825 non-null  object 
 1   main_category    533825 non-null  object 
 2   sub_category     533825 non-null  object 
 3   image            533825 non-null  object 
 4   link             533825 non-null  object 
 5   ratings          533825 non-null  float64
 6   no_of_ratings    533825 non-null  Int64  
 7   discount_price   533825 non-null  float64
 8   actual_price     533825 non-null  float64
 9   discount_amount  533825 non-null  float64
dtypes: Int64(1), float64(4), object(5)
memory usage: 45.3+ MB


**Handling duplicate data: further checking:**

Upon further inspection, I noticed some entries have identical names and values but differ in category. These records need to be addressed to ensure the analysis is valid.

In [15]:
#df_cleaned_v2 = df_cleaned_v1.drop_duplicates(subset='name', keep="first")
df_cleaned_v2 = df_cleaned_v1.groupby('name').first().reset_index()

**Exporting cleaned dataset: data is ready for analysis:**

In [16]:
#setting export path and csv file name
export_path = (r"C:\Users\user\OneDrive\RFA _Personal Files\02. COURSE\Purwadhika_Data Engineering\Purwadhika_VS\01_Personal_Projects\01_ETL01\data_cleaned\data_products_cleaned.csv")
               
df_cleaned_v2.to_csv(export_path, index=False)